# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [17]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
customer_product_quantity = data.groupby(['CustomerID', 'ProductName'])['Quantity'].sum().reset_index()
customer_product_quantity

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
product_customer_matrix = pd.pivot_table(customer_product_quantity, 
                                         index='ProductName', 
                                         columns='CustomerID', 
                                         values='Quantity', 
                                         fill_value=0)
product_customer_matrix

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
"Yogurt - Blueberry, 175 Gr",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
pairwise_distances = pdist(product_customer_matrix.values, metric='euclidean')

customer_similarity_matrix = pd.DataFrame(squareform(pairwise_distances), 
                                          index=product_customer_matrix.index, 
                                          columns=product_customer_matrix.index)

customer_similarity_matrix

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.000000,279.996429,253.669076,238.512054,242.810626,275.550358,264.474952,257.029181,247.519696,247.570192,...,262.608073,265.744238,258.980694,246.704276,259.474469,247.382295,254.911749,255.671664,258.091069,278.478006
"Appetizer - Mini Egg Roll, Shrimp",279.996429,0.000000,271.499540,269.455006,269.838841,292.006849,286.438475,255.432966,262.922042,285.854858,...,273.822935,286.251638,274.952723,265.625676,273.830239,284.024647,269.896276,265.736712,279.555004,286.806555
Appetizer - Mushroom Tart,253.669076,271.499540,0.000000,235.210544,258.810742,262.891613,260.862032,241.722982,242.317973,270.272085,...,254.108245,270.510628,261.306334,225.341962,239.681038,242.235423,243.191283,245.275355,253.015810,269.436449
Appetizer - Sausage Rolls,238.512054,269.455006,235.210544,0.000000,244.235542,250.659131,261.600841,238.872351,249.110417,259.613174,...,253.978346,258.530462,251.414001,218.753286,243.530286,257.297493,243.991803,247.737765,256.894920,267.985074
Apricots - Dried,242.810626,269.838841,258.810742,244.235542,0.000000,265.343174,267.675176,244.497444,255.759653,263.711964,...,264.979244,271.556624,257.487864,235.244554,259.703677,267.968282,247.864883,251.855117,261.400077,264.497637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,247.382295,284.024647,242.235423,257.297493,267.968282,267.305069,267.804033,246.519776,249.843951,263.797271,...,248.586001,274.105819,269.697979,244.804003,255.391856,0.000000,258.793354,262.811720,265.768696,276.134025
Yoghurt Tubes,254.911749,269.896276,243.191283,243.991803,247.864883,266.514540,253.911402,244.413584,251.809452,262.562374,...,253.331798,264.329340,259.027026,243.004115,242.901214,258.793354,0.000000,256.970815,259.844184,269.903687
"Yogurt - Blueberry, 175 Gr",255.671664,265.736712,245.275355,247.737765,251.855117,257.243076,263.068432,246.734675,245.833277,269.653481,...,257.648986,265.126385,259.150535,242.365427,257.105037,262.811720,256.970815,0.000000,264.142007,268.283432


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [12]:

similarities_with_specific_customer = data.loc['CustomerID']

sorted_similarities = similarities_with_specific_customer.sort_values()

top_5_similar_customers = sorted_similarities.iloc[1:6]

print("Top 5 most similar customers for CustomerID",'CustomerID', ":")
print(top_5_similar_customers)


KeyError: 'CustomerID'

In [11]:
data.columns

Index(['CustomerID', 'FirstName', 'LastName', 'SalesID', 'ProductID',
       'ProductName', 'Quantity'],
      dtype='object')

In [15]:
data.loc[data['CustomerID'] == specific_customer_id]

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
527,33,Lindsay,Santana,1899667,432,Beer - Original Organic Lager,1
1017,33,Lindsay,Santana,6509275,417,"Lamb - Pieces, Diced",1
3012,33,Lindsay,Santana,5638266,214,French Pastry - Mini Chocolate,1
3476,33,Lindsay,Santana,4280329,284,Beer - Rickards Red,1
3795,33,Lindsay,Santana,5869069,342,Mangoes,1
...,...,...,...,...,...,...,...
61648,33,Lindsay,Santana,6507033,300,"Beans - Kidney, Canned",1
62883,33,Lindsay,Santana,4150957,319,Towels - Paper / Kraft,1
63171,33,Lindsay,Santana,3056733,187,"Pepper - Black, Whole",1
66163,33,Lindsay,Santana,3179606,296,Wine - Valpolicella Masi,1


In [16]:
print(data.columns)

# Check if 'CustomerID' exists in the DataFrame
if 'CustomerID' in data.columns:
    # Inspect unique values in the 'CustomerID' column
    print(data['CustomerID'].unique())
else:
    print("'CustomerID' column does not exist in the DataFrame.")

Index(['CustomerID', 'FirstName', 'LastName', 'SalesID', 'ProductID',
       'ProductName', 'Quantity'],
      dtype='object')
[61288 77352 40094 23548 78981 83106 11253 35107 15088 26031 86595 46715
 91013 69931 19753  9546 10224 72846 28976  3267 19260 76530 30196 48818
 15166 11443 79060 82597 62411 42611 45420 83495 47122 30815 53198 18591
 62034  1428  2187 55162 22345 81876 48641 81034 29681 87947 28867 69967
 94420 83158 84364 44742  7772 47303 75791 77756 97324 13912 52700 21734
  6817 21095 28875 89588  1034 54827 29287 11102 52155 50748 88552 80938
 60481 92168 66232 64348 37510 44408 89254   477 51483 97201 42157 52091
 86065 18027 93882 42303 32010 53997 63033 76475 98185 84779 82146 54495
  3903 79854 25449 85496 95017 33759 43652 35873 97769   891 85161 25327
 25624 68416 44090 71595 68336 38497 10016 36751 58421 45498 50774 79997
 45313 55715 40901  1577  3909 18480 41405 85552   412 59399 87849 26458
 13354 92673 45759 55930 62184 70512 51451 46046  1754  2915 81591 781

In [19]:
import numpy as np

# Define the specific CustomerID
specific_customer_id = 

# Calculate Euclidean distances between the specific CustomerID and all other customers
distances_to_specific_customer = customer_similarity_matrix[specific_customer_id]

# Sort the distances in ascending order
sorted_distances = np.argsort(distances_to_specific_customer)

# Exclude the first element (similarity with itself) and select the top 5 most similar customers
top_5_similar_customers = sorted_distances[1:6]

# Display the top 5 most similar customers
print("Top 5 most similar customers for CustomerID", specific_customer_id, ":")
print(top_5_similar_customers)


KeyError: 7606

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [21]:
similar_customer_ids = [list_of_similar_CustomerIDs_here]

similar_customer_records = customer_product_quantity[customer_product_quantity['CustomerID'].isin(similar_customer_ids)]

similar_customer_records


NameError: name 'list_of_similar_CustomerIDs_here' is not defined

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [20]:
similar_customer_records = similar_customer_records[similar_customer_records['CustomerID'].isin(top_5_similar_customers)]

product_purchase_totals = similar_customer_records.groupby('ProductName')['Quantity'].sum().reset_index()

product_purchase_totals = product_purchase_totals.sort_values(by='Quantity', ascending=False)

print(product_purchase_totals)


NameError: name 'similar_customer_records' is not defined

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [ ]:

merged_df = pd.merge(product_purchase_totals, product_customer_matrix, on='ProductName', how='left')

customer_not_purchased = merged_df[merged_df[specific_customer_id] == 0]

top_5_recommendations = customer_not_purchased.head(5)

top_5_recommendations


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.